In [29]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import os
import csv

In [30]:
# Load a pre-trained ResNet-18 model
resnet18 = models.resnet18(pretrained=True)
# Remove the classification layer (the final fully connected layer)
encoder = nn.Sequential(*list(resnet18.children())[:-1])

# Define a transformation to preprocess the images
data_transform = transforms.Compose([
    transforms.Resize(256),  # Resize the image to 256x256
    transforms.CenterCrop(224),  # Crop a 224x224 region from the center
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Load and preprocess an example image
image_path = 'NPRVideoFrames\\2nd Batch Of Aid Reaches Gaza As Israeli Air Strikes Intensify  NPR News Now\\2nd Batch Of Aid Reaches Gaza As Israeli Air Strikes Intensify  NPR News Now_frame_0000.jpg'
image = Image.open(image_path)
image = data_transform(image)  # Apply the transformation

# Encode the image to get the feature vector
with torch.no_grad():
    feature_vector = encoder(image.unsqueeze(0))  # Add a batch dimension

# The 'feature_vector' now contains the encoded representation of the image
print(feature_vector)


c:\Users\thaou\miniconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\thaou\miniconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


tensor([[[[0.9763]],

         [[0.9506]],

         [[0.9476]],

         [[1.0836]],

         [[0.8962]],

         [[0.8548]],

         [[0.9593]],

         [[1.1899]],

         [[0.9256]],

         [[0.9706]],

         [[0.9623]],

         [[0.9061]],

         [[0.9554]],

         [[0.9234]],

         [[0.9213]],

         [[0.9215]],

         [[0.8939]],

         [[1.4169]],

         [[0.8491]],

         [[0.8307]],

         [[0.9420]],

         [[1.0892]],

         [[0.9291]],

         [[1.0042]],

         [[0.8377]],

         [[0.9820]],

         [[1.0050]],

         [[0.9891]],

         [[0.8418]],

         [[0.9186]],

         [[0.9974]],

         [[0.9052]],

         [[0.9385]],

         [[0.8751]],

         [[0.8991]],

         [[0.8731]],

         [[0.9402]],

         [[0.9425]],

         [[0.8776]],

         [[0.9898]],

         [[0.8513]],

         [[0.8553]],

         [[0.8324]],

         [[0.9433]],

         [[0.9003]],

         [

In [31]:
# Load a pre-trained ResNet-18 model
resnet18 = models.resnet18(pretrained=True)
# Remove the classification layer (the final fully connected layer)
encoder = nn.Sequential(*list(resnet18.children())[:-1])

# Define a transformation to preprocess the images
data_transform = transforms.Compose([
    transforms.Resize(256), 
    transforms.CenterCrop(224),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [32]:
# Relevant folders
frames_folder = 'NPRVideoFrames'
embeddings_folder = 'NPREmbeddings'
os.makedirs(embeddings_folder, exist_ok=True)

# Loop through all subfolders in the frames folder
for subfolder in os.listdir(frames_folder):
    subfolder_path = os.path.join(frames_folder, subfolder)

    if os.path.isdir(subfolder_path):
        # List to store tuples of frame names and feature vectors for frames in the same video
        video_feature_list = []

        for frame in os.listdir(subfolder_path):
            frame_path = os.path.join(subfolder_path, frame)
            image = Image.open(frame_path)
            image = data_transform(image)

            frame_name = frame.replace("  NPR News Now", "")
            frame_name = frame_name.replace(".jpg", "")  
            frame_name = frame_name.replace(" ", "_")  

            # ENCODE THE IMAGE
            # -- The 'feature_vector' contains the encoded representation of the image
            with torch.no_grad():
                feature_vector = encoder(image.unsqueeze(0))

            # -- Flatten the tensor and convert to NumPy array
            flat_vector = feature_vector.view(-1).numpy()

            # -- Append the frame name and feature vector tuple to the list for the current video
            video_feature_list.append((frame_name, flat_vector))

        # EMBEDDING STORAGE IN CSV FILE
        # -- Define the CSV file path
        video_name = frame_name[:-11].replace("_", " ")
        csv_file_path = os.path.join(embeddings_folder, f"{video_name}.csv")

        # -- Write the data to the CSV file: define the column names and fill in the values
        with open(csv_file_path, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Frame_Name'] + [f'Feature_{i}' for i in range(len(flat_vector))])

            for frame_name, flat_vector in video_feature_list:
                writer.writerow([frame_name] + flat_vector.tolist())
            
            print(video_name)

2000 Bodies Recovered After Dam Bursts In Derna Libya
2nd Batch Of Aid Reaches Gaza As Israeli Air Strikes Intensify
8 Face Off In GOP Presidential Debate Tonight
American Cave Explorer Rescued In Turkey
Biden To Walk UAW Picket Line In Michigan Today
Biden Visits Israel As Gaza Reels From Deadly Hospital Explosion
Biden Zelenskyy Address UNs General Assembly
Blinken Announces 1 Billion In New Aid To Ukraine
Blinken Visits Israel As Palestinian Israeli Deaths Mount
Candidates Spar Over The Economy Abortion In Milwaukee Debate
Cleanup Begins After Hilary Pummels Southern California
Climate Week Protests Draw Thousands In New York City
England Advances To World Cup Finals 106 Dead In Maui
Florida Communities Prepare For Delayed Flooding
Former Proud Boys Leader Enrique Tarrio To Be Sentenced Today
Gaetz Calls For Vote To Oust House Speaker Kevin McCarthy
Gaetz Leads 7 Republicans House Democrats In Ousting McCarthy
Hilary Continues To Shower Northern Mexico Southern California
Hong Kong 